In [ ]:
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np




#Importing all the relevant library
%matplotlib inline
import h5py, os
#from functions import transforms as T
#from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
#from functions import transforms as T 
#from functions.subsample import MaskFunc
from PIL import Image

In [ ]:
def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    data_path = [train_data_path, val_data_path]
      
    for i in range(len(data_path)):

        data_list[train_and_val[i]] = []
        
        which_data_path = data_path[i]
        tr = 0
        te = 0
        alfa = 0
        for fname in sorted(os.listdir(which_data_path + '/images')):
            
            subject_data_path = os.path.join(which_data_path + '/images', fname)
                     
            if not os.path.isfile(subject_data_path): continue 
            
            im_frame = Image.open(subject_data_path)

            #get information from text file
            # this will return a tuple of root and extension
            split_tup = os.path.splitext(fname)

  
            # extract the file name and extension
            file_name = split_tup[0]
            f = open(os.path.join(which_data_path + '/texts', file_name) + '.txt', 'r')

            for line in f.readlines():
                fields = line.split(',')
                tr = int(fields[1])
                te = int(fields[2])
                alfa = int(fields[3])
                f.close()
                
            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(im_frame, fname, tr)]
    
    return data_list

In [ ]:
data_list = load_data_path ("train data path", "valid data path")

In [ ]:
train_data = data_list['train']
val_data = data_list['val']

In [ ]:
train_df = pd.DataFrame(train_data, columns=['image', 'fnames', 'labels'])
val_df = pd.DataFrame(val_data, columns=['image', 'fnames', 'labels'])
labels = train_df.labels.unique()

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
      dataframe=train_df,
      directory="./data/images",
      x_col="fnames",
      y_col="labels",
      batch_size=32,
      seed=42,
      shuffle=True,
      class_mode="categorical",
      classes=labels,
      target_size=(100,100))



valid_generator=test_datagen.flow_from_dataframe(
      dataframe=val_df,
      directory="./data/images",
      x_col="fnames",
      y_col="labels",
      batch_size=32,
      seed=42,
      shuffle=True,
      class_mode="categorical",
      classes=labels,
      target_size=(100,100))


In [ ]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=df[1900:],
      directory="./data/images",
      x_col="fnames",
      batch_size=1,
      seed=42,
      shuffle=False,
      class_mode=None,
      target_size=(100,100))

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
pred_bool = (pred >0.5)

In [ ]:
predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(label[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)